# Basic model with static forcing

In [ ]:
# | include: false
import os

os.environ["USE_PYGEOS"] = "0"

In [ ]:
from pathlib import Path

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ribasim

Setup the basins:

In [ ]:
profile = pd.DataFrame(
    data={
        "node_id": [1, 1, 3, 3, 6, 6, 9, 9],
        "area": [0.01, 1000.0] * 4,
        "level": [0.0, 1.0] * 4,
    }
)

# Convert steady forcing to m/s
# 2 mm/d precipitation, 1 mm/d evaporation
seconds_in_day = 24 * 3600
precipitation = 0.002 / seconds_in_day
evaporation = 0.001 / seconds_in_day

static = pd.DataFrame(
    data={
        "node_id": [0],
        "potential_evaporation": [evaporation],
        "precipitation": [precipitation],
    }
)
static = static.iloc[[0, 0, 0, 0]]
static["node_id"] = [1, 3, 6, 9]

basin = ribasim.Basin(profile=profile, static=static)

Setup linear resistance:

In [ ]:
linear_resistance = ribasim.LinearResistance(
    static=pd.DataFrame(
        data={"node_id": [10, 12], "resistance": [5e3, (3600.0 * 24) / 100.0]}
    )
)

Setup Manning resistance:

In [ ]:
manning_resistance = ribasim.ManningResistance(
    static=pd.DataFrame(
        data={
            "node_id": [2],
            "length": [900.0],
            "manning_n": [0.04],
            "profile_width": [6.0],
            "profile_slope": [3.0],
        }
    )
)

Set up a rating curve node:

In [ ]:
# Discharge: lose 1% of storage volume per day at storage = 1000.0.
q1000 = 1000.0 * 0.01 / seconds_in_day

rating_curve = ribasim.TabulatedRatingCurve(
    static=pd.DataFrame(
        data={
            "node_id": [4, 4],
            "level": [0.0, 1.0],
            "flow_rate": [0.0, q1000],
        }
    )
)

Setup fractional flows:

In [ ]:
fractional_flow = ribasim.FractionalFlow(
    static=pd.DataFrame(
        data={
            "node_id": [5, 8, 13],
            "fraction": [0.3, 0.6, 0.1],
        }
    )
)

Setup pump:

In [ ]:
pump = ribasim.Pump(
    static=pd.DataFrame(
        data={
            "node_id": [7],
            "flow_rate": [0.5 / 3600],
        }
    )
)

Setup level boundary:

In [ ]:
level_boundary = ribasim.LevelBoundary(
    static=pd.DataFrame(
        data={
            "node_id": [11, 17],
            "level": [0.5, 1.5],
        }
    )
)

Setup flow boundary:

In [ ]:
flow_boundary = ribasim.FlowBoundary(
    static=pd.DataFrame(
        data={
            "node_id": [15, 16],
            "flow_rate": [1e-4, 1e-4],
        }
    )
)

Setup terminal:

In [ ]:
terminal = ribasim.Terminal(
    static=pd.DataFrame(
        data={
            "node_id": [14],
        }
    )
)

Set up the nodes:

In [ ]:
xy = np.array(
    [
        (0.0, 0.0),  # 1: Basin,
        (1.0, 0.0),  # 2: ManningResistance
        (2.0, 0.0),  # 3: Basin
        (3.0, 0.0),  # 4: TabulatedRatingCurve
        (3.0, 1.0),  # 5: FractionalFlow
        (3.0, 2.0),  # 6: Basin
        (4.0, 1.0),  # 7: Pump
        (4.0, 0.0),  # 8: FractionalFlow
        (5.0, 0.0),  # 9: Basin
        (6.0, 0.0),  # 10: LinearResistance
        (2.0, 2.0),  # 11: LevelBoundary
        (2.0, 1.0),  # 12: LinearResistance
        (3.0, -1.0),  # 13: FractionalFlow
        (3.0, -2.0),  # 14: Terminal
        (3.0, 3.0),  # 15: FlowBoundary
        (0.0, 1.0),  # 16: FlowBoundary
        (6.0, 1.0),  # 17: LevelBoundary
    ]
)
node_xy = gpd.points_from_xy(x=xy[:, 0], y=xy[:, 1])

node_id, node_type = ribasim.Node.node_ids_and_types(
    basin,
    manning_resistance,
    rating_curve,
    pump,
    fractional_flow,
    linear_resistance,
    level_boundary,
    flow_boundary,
    terminal,
)

# Make sure the feature id starts at 1: explicitly give an index.
node = ribasim.Node(
    df=gpd.GeoDataFrame(
        data={"node_type": node_type},
        index=pd.Index(node_id, name="fid"),
        geometry=node_xy,
        crs="EPSG:28992",
    )
)

Setup the edges:

In [ ]:
from_id = np.array(
    [1, 2, 3, 4, 4, 5, 6, 8, 7, 9, 11, 12, 4, 13, 15, 16, 10], dtype=np.int64
)
to_id = np.array(
    [2, 3, 4, 5, 8, 6, 7, 9, 9, 10, 12, 3, 13, 14, 6, 1, 17], dtype=np.int64
)
lines = node.geometry_from_connectivity(from_id, to_id)
edge = ribasim.Edge(
    df=gpd.GeoDataFrame(
        data={
            "from_node_id": from_id,
            "to_node_id": to_id,
            "edge_type": len(from_id) * ["flow"],
        },
        geometry=lines,
        crs="EPSG:28992",
    )
)

Setup a model:

In [ ]:
model = ribasim.Model(
    network=ribasim.Network(
        node=node,
        edge=edge,
    ),
    basin=basin,
    level_boundary=level_boundary,
    flow_boundary=flow_boundary,
    pump=pump,
    linear_resistance=linear_resistance,
    manning_resistance=manning_resistance,
    tabulated_rating_curve=rating_curve,
    fractional_flow=fractional_flow,
    terminal=terminal,
    starttime="2020-01-01 00:00:00",
    endtime="2021-01-01 00:00:00",
)

Let's take a look at the model:

In [ ]:
model.plot()

Write the model to a TOML and GeoPackage:

In [ ]:
datadir = Path("data")
model.write(datadir / "basic/ribasim.toml")

# Update the basic model with transient forcing

This assumes you have already created the basic model with static forcing.

In [ ]:
import numpy as np
import pandas as pd
import ribasim
import xarray as xr

In [ ]:
model = ribasim.Model(filepath=datadir / "basic/ribasim.toml")

In [ ]:
time = pd.date_range(model.starttime, model.endtime)
day_of_year = time.day_of_year.to_numpy()
seconds_per_day = 24 * 60 * 60
evaporation = (
    (-1.0 * np.cos(day_of_year / 365.0 * 2 * np.pi) + 1.0) * 0.0025 / seconds_per_day
)
rng = np.random.default_rng(seed=0)
precipitation = (
    rng.lognormal(mean=-1.0, sigma=1.7, size=time.size) * 0.001 / seconds_per_day
)

We'll use xarray to easily broadcast the values.

In [ ]:
timeseries = (
    pd.DataFrame(
        data={
            "node_id": 1,
            "time": pd.date_range(model.starttime, model.endtime),
            "drainage": 0.0,
            "potential_evaporation": evaporation,
            "infiltration": 0.0,
            "precipitation": precipitation,
            "urban_runoff": 0.0,
        }
    )
    .set_index("time")
    .to_xarray()
)

basin_ids = model.basin.static.df["node_id"].to_numpy()
basin_nodes = xr.DataArray(
    np.ones(len(basin_ids)), coords={"node_id": basin_ids}, dims=["node_id"]
)
forcing = (timeseries * basin_nodes).to_dataframe().reset_index()

In [ ]:
state = pd.DataFrame(
    data={
        "node_id": basin_ids,
        "level": 1.4,
    }
)

In [ ]:
model.basin.time.df = forcing
model.basin.state.df = state

In [ ]:
model.write(datadir / "basic_transient/ribasim.toml")

In [ ]:
# | include: false
from subprocess import run

run(
    [
        "julia",
        "--project=../../core",
        "--eval",
        f'using Ribasim; Ribasim.main("{datadir.as_posix()}/basic_transient/ribasim.toml")',
    ],
    check=True,
)

Now run the model with `ribasim basic_transient/ribasim.toml`.
After running the model, read back the results:

In [ ]:
df_basin = pd.read_feather(datadir / "basic_transient/results/basin.arrow")
df_basin_wide = df_basin.pivot_table(
    index="time", columns="node_id", values=["storage", "level"]
)
df_basin_wide["level"].plot()

In [ ]:
df_flow = pd.read_feather(datadir / "basic_transient/results/flow.arrow")
df_flow["edge"] = list(zip(df_flow.from_node_id, df_flow.to_node_id))
df_flow["flow_m3d"] = df_flow.flow_rate * 86400
ax = df_flow.pivot_table(index="time", columns="edge", values="flow_m3d").plot()
ax.legend(bbox_to_anchor=(1.3, 1), title="Edge")

In [ ]:
type(df_flow)

# Model with discrete control

The model constructed below consists of a single basin which slowly drains trough a `TabulatedRatingCurve`, but is held within a range around a target level (setpoint) by two connected pumps. These two pumps behave like a reversible pump. When pumping can be done in only one direction, and the other direction is only possible under gravity, use an Outlet for that direction.

Set up the nodes:

In [ ]:
xy = np.array(
    [
        (0.0, 0.0),  # 1: Basin
        (1.0, 1.0),  # 2: Pump
        (1.0, -1.0),  # 3: Pump
        (2.0, 0.0),  # 4: LevelBoundary
        (-1.0, 0.0),  # 5: TabulatedRatingCurve
        (-2.0, 0.0),  # 6: Terminal
        (1.0, 0.0),  # 7: DiscreteControl
    ]
)

node_xy = gpd.points_from_xy(x=xy[:, 0], y=xy[:, 1])

node_type = [
    "Basin",
    "Pump",
    "Pump",
    "LevelBoundary",
    "TabulatedRatingCurve",
    "Terminal",
    "DiscreteControl",
]

# Make sure the feature id starts at 1: explicitly give an index.
node = ribasim.Node(
    df=gpd.GeoDataFrame(
        data={"node_type": node_type},
        index=pd.Index(np.arange(len(xy)) + 1, name="fid"),
        geometry=node_xy,
        crs="EPSG:28992",
    )
)

Setup the edges:

In [ ]:
from_id = np.array([1, 3, 4, 2, 1, 5, 7, 7], dtype=np.int64)
to_id = np.array([3, 4, 2, 1, 5, 6, 2, 3], dtype=np.int64)

edge_type = 6 * ["flow"] + 2 * ["control"]

lines = node.geometry_from_connectivity(from_id, to_id)
edge = ribasim.Edge(
    df=gpd.GeoDataFrame(
        data={"from_node_id": from_id, "to_node_id": to_id, "edge_type": edge_type},
        geometry=lines,
        crs="EPSG:28992",
    )
)

Setup the basins:

In [ ]:
profile = pd.DataFrame(
    data={
        "node_id": [1, 1],
        "area": [1000.0, 1000.0],
        "level": [0.0, 1.0],
    }
)

state = pd.DataFrame(data={"node_id": [1], "level": [20.0]})

basin = ribasim.Basin(profile=profile, state=state)

Setup the discrete control:

In [ ]:
condition = pd.DataFrame(
    data={
        "node_id": 3 * [7],
        "listen_feature_id": 3 * [1],
        "variable": 3 * ["level"],
        "greater_than": [5.0, 10.0, 15.0],  # min, setpoint, max
    }
)

logic = pd.DataFrame(
    data={
        "node_id": 5 * [7],
        "truth_state": ["FFF", "U**", "T*F", "**D", "TTT"],
        "control_state": ["in", "in", "none", "out", "out"],
    }
)

discrete_control = ribasim.DiscreteControl(condition=condition, logic=logic)

The above control logic can be summarized as follows:
- If the level gets above the maximum, activate the control state "out" until the setpoint is reached;
- If the level gets below the minimum, active the control state "in" until the setpoint is reached;
- Otherwise activate the control state "none".

Setup the pump:

In [ ]:
pump = ribasim.Pump(
    static=pd.DataFrame(
        data={
            "node_id": 3 * [2] + 3 * [3],
            "control_state": 2 * ["none", "in", "out"],
            "flow_rate": [0.0, 2e-3, 0.0, 0.0, 0.0, 2e-3],
        }
    )
)

The pump data defines the following:

Control state | Pump #2 flow rate (m/s)| Pump #3 flow rate (m/s)
--------------|------------------------|------------------
"none"        | 0.0                    | 0.0
"in"          | 2e-3                   | 0.0
"out"         | 0.0                    | 2e-3

Setup the level boundary:

In [ ]:
level_boundary = ribasim.LevelBoundary(
    static=pd.DataFrame(data={"node_id": [4], "level": [10.0]})
)

Setup the rating curve:

In [ ]:
rating_curve = ribasim.TabulatedRatingCurve(
    static=pd.DataFrame(
        data={"node_id": 2 * [5], "level": [2.0, 15.0], "flow_rate": [0.0, 1e-3]}
    )
)

Setup the terminal:

In [ ]:
terminal = ribasim.Terminal(static=pd.DataFrame(data={"node_id": [6]}))

Setup a model:

In [ ]:
model = ribasim.Model(
    network=ribasim.Network(
        node=node,
        edge=edge,
    ),
    basin=basin,
    pump=pump,
    level_boundary=level_boundary,
    tabulated_rating_curve=rating_curve,
    terminal=terminal,
    discrete_control=discrete_control,
    starttime="2020-01-01 00:00:00",
    endtime="2021-01-01 00:00:00",
)

Let’s take a look at the model:

In [ ]:
model.plot()

Listen edges are plotted with a dashed line since they are not present in the "Edge / static" schema but only in the "Control / condition" schema.

In [ ]:
datadir = Path("data")
model.write(datadir / "level_setpoint_with_minmax/ribasim.toml")

In [ ]:
# | include: false
from subprocess import run

run(
    [
        "julia",
        "--project=../../core",
        "--eval",
        f'using Ribasim; Ribasim.main("{datadir.as_posix()}/level_setpoint_with_minmax/ribasim.toml")',
    ],
    check=True,
)

Now run the model with `level_setpoint_with_minmax/ribasim.toml`.
After running the model, read back the results:

In [ ]:
from matplotlib.dates import date2num

df_basin = pd.read_feather(datadir / "level_setpoint_with_minmax/results/basin.arrow")
df_basin_wide = df_basin.pivot_table(
    index="time", columns="node_id", values=["storage", "level"]
)

ax = df_basin_wide["level"].plot()

greater_than = model.discrete_control.condition.df.greater_than

ax.hlines(
    greater_than,
    df_basin.time[0],
    df_basin.time.max(),
    lw=1,
    ls="--",
    color="k",
)

df_control = pd.read_feather(
    datadir / "level_setpoint_with_minmax/results/control.arrow"
)

y_min, y_max = ax.get_ybound()
ax.fill_between(df_control.time[:2], 2 * [y_min], 2 * [y_max], alpha=0.2, color="C0")
ax.fill_between(df_control.time[2:4], 2 * [y_min], 2 * [y_max], alpha=0.2, color="C0")

ax.set_xticks(
    date2num(df_control.time).tolist(),
    df_control.control_state.tolist(),
    rotation=50,
)

ax.set_yticks(greater_than, ["min", "setpoint", "max"])
ax.set_ylabel("level")
plt.show()

The highlighted regions show where a pump is active.

Let's print an overview of what happened with control:

In [ ]:
model.print_discrete_control_record(
    datadir / "level_setpoint_with_minmax/results/control.arrow"
)

Note that crossing direction specific truth states (containing "U", "D") are not present in this overview even though they are part of the control logic. This is because in the control logic for this model these truth states are only used to sustain control states, while the overview only shows changes in control states.

# Model with PID control

Set up the nodes:

In [ ]:
xy = np.array(
    [
        (0.0, 0.0),  # 1: FlowBoundary
        (1.0, 0.0),  # 2: Basin
        (2.0, 0.5),  # 3: Pump
        (3.0, 0.0),  # 4: LevelBoundary
        (1.5, 1.0),  # 5: PidControl
        (2.0, -0.5),  # 6: outlet
        (1.5, -1.0),  # 7: PidControl
    ]
)

node_xy = gpd.points_from_xy(x=xy[:, 0], y=xy[:, 1])

node_type = [
    "FlowBoundary",
    "Basin",
    "Pump",
    "LevelBoundary",
    "PidControl",
    "Outlet",
    "PidControl",
]

# Make sure the feature id starts at 1: explicitly give an index.
node = ribasim.Node(
    df=gpd.GeoDataFrame(
        data={"node_type": node_type},
        index=pd.Index(np.arange(len(xy)) + 1, name="fid"),
        geometry=node_xy,
        crs="EPSG:28992",
    )
)

Setup the edges:

In [ ]:
from_id = np.array([1, 2, 3, 4, 6, 5, 7], dtype=np.int64)
to_id = np.array([2, 3, 4, 6, 2, 3, 6], dtype=np.int64)

lines = node.geometry_from_connectivity(from_id, to_id)
edge = ribasim.Edge(
    df=gpd.GeoDataFrame(
        data={
            "from_node_id": from_id,
            "to_node_id": to_id,
            "edge_type": 5 * ["flow"] + 2 * ["control"],
        },
        geometry=lines,
        crs="EPSG:28992",
    )
)

Setup the basins:

In [ ]:
profile = pd.DataFrame(
    data={"node_id": [2, 2], "level": [0.0, 1.0], "area": [1000.0, 1000.0]}
)

state = pd.DataFrame(
    data={
        "node_id": [2],
        "level": [6.0],
    }
)

basin = ribasim.Basin(profile=profile, state=state)

Setup the pump:

In [ ]:
pump = ribasim.Pump(
    static=pd.DataFrame(
        data={
            "node_id": [3],
            "flow_rate": [0.0],  # Will be overwritten by PID controller
        }
    )
)

Setup the outlet:

In [ ]:
outlet = ribasim.Outlet(
    static=pd.DataFrame(
        data={
            "node_id": [6],
            "flow_rate": [0.0],  # Will be overwritten by PID controller
        }
    )
)

Setup flow boundary:

In [ ]:
flow_boundary = ribasim.FlowBoundary(
    static=pd.DataFrame(data={"node_id": [1], "flow_rate": [1e-3]})
)

Setup flow boundary:

In [ ]:
level_boundary = ribasim.LevelBoundary(
    static=pd.DataFrame(
        data={
            "node_id": [4],
            "level": [1.0],  # Not relevant
        }
    )
)

Setup PID control:

In [ ]:
pid_control = ribasim.PidControl(
    time=pd.DataFrame(
        data={
            "node_id": 4 * [5, 7],
            "time": [
                "2020-01-01 00:00:00",
                "2020-01-01 00:00:00",
                "2020-05-01 00:00:00",
                "2020-05-01 00:00:00",
                "2020-07-01 00:00:00",
                "2020-07-01 00:00:00",
                "2020-12-01 00:00:00",
                "2020-12-01 00:00:00",
            ],
            "listen_node_id": 4 * [2, 2],
            "target": [5.0, 5.0, 5.0, 5.0, 7.5, 7.5, 7.5, 7.5],
            "proportional": 4 * [-1e-3, 1e-3],
            "integral": 4 * [-1e-7, 1e-7],
            "derivative": 4 * [0.0, 0.0],
        }
    )
)

Note that the coefficients for the pump and the outlet are equal in magnitude but opposite in sign. This way the pump and the outlet equally work towards the same goal, while having opposite effects on the controlled basin due to their connectivity to this basin.

Setup a model:

In [ ]:
model = ribasim.Model(
    network=ribasim.Network(
        node=node,
        edge=edge,
    ),
    basin=basin,
    flow_boundary=flow_boundary,
    level_boundary=level_boundary,
    pump=pump,
    outlet=outlet,
    pid_control=pid_control,
    starttime="2020-01-01 00:00:00",
    endtime="2020-12-01 00:00:00",
)

Let's take a look at the model:

In [ ]:
model.plot()

Write the model to a TOML and GeoPackage:

In [ ]:
datadir = Path("data")
model.write(datadir / "pid_control/ribasim.toml")

In [ ]:
# | include: false
from subprocess import run

run(
    [
        "julia",
        "--project=../../core",
        "--eval",
        f'using Ribasim; Ribasim.main("{datadir.as_posix()}/pid_control/ribasim.toml")',
    ],
    check=True,
)

Now run the model with `ribasim pid_control/ribasim.toml`.
After running the model, read back the results:

In [ ]:
from matplotlib.dates import date2num

df_basin = pd.read_feather(datadir / "pid_control/results/basin.arrow")
df_basin_wide = df_basin.pivot_table(
    index="time", columns="node_id", values=["storage", "level"]
)
ax = df_basin_wide["level"].plot()
ax.set_ylabel("level [m]")

# Plot target level
level_demands = model.pid_control.time.df.target.to_numpy()[:4]
times = date2num(model.pid_control.time.df.time)[:4]
ax.plot(times, level_demands, color="k", ls=":", label="target level")
pass

# Model with allocation (user demand)

Setup the nodes:

In [ ]:
xy = np.array(
    [
        (0.0, 0.0),  # 1: FlowBoundary
        (1.0, 0.0),  # 2: Basin
        (1.0, 1.0),  # 3: User
        (2.0, 0.0),  # 4: LinearResistance
        (3.0, 0.0),  # 5: Basin
        (3.0, 1.0),  # 6: User
        (4.0, 0.0),  # 7: TabulatedRatingCurve
        (4.5, 0.0),  # 8: FractionalFlow
        (4.5, 0.5),  # 9: FractionalFlow
        (5.0, 0.0),  # 10: Terminal
        (4.5, 0.25),  # 11: DiscreteControl
        (4.5, 1.0),  # 12: Basin
        (5.0, 1.0),  # 13: User
    ]
)
node_xy = gpd.points_from_xy(x=xy[:, 0], y=xy[:, 1])

node_type = [
    "FlowBoundary",
    "Basin",
    "User",
    "LinearResistance",
    "Basin",
    "User",
    "TabulatedRatingCurve",
    "FractionalFlow",
    "FractionalFlow",
    "Terminal",
    "DiscreteControl",
    "Basin",
    "User",
]

# All nodes belong to allocation network id 1
node = ribasim.Node(
    df=gpd.GeoDataFrame(
        data={"node_type": node_type, "subnetwork_id": 1},
        index=pd.Index(np.arange(len(xy)) + 1, name="fid"),
        geometry=node_xy,
        crs="EPSG:28992",
    )
)

Setup the edges:

In [ ]:
from_id = np.array(
    [1, 2, 2, 4, 5, 5, 7, 3, 6, 7, 8, 9, 12, 13, 11, 11],
    dtype=np.int64,
)
to_id = np.array(
    [2, 3, 4, 5, 6, 7, 8, 2, 5, 9, 10, 12, 13, 10, 8, 9],
    dtype=np.int64,
)
# Denote the first edge, 1 => 2, as a source edge for
# allocation network 1
subnetwork_id = len(from_id) * [None]
subnetwork_id[0] = 1
lines = node.geometry_from_connectivity(from_id, to_id)
edge = ribasim.Edge(
    df=gpd.GeoDataFrame(
        data={
            "from_node_id": from_id,
            "to_node_id": to_id,
            "edge_type": (len(from_id) - 2) * ["flow"] + 2 * ["control"],
            "subnetwork_id": subnetwork_id,
        },
        geometry=lines,
        crs="EPSG:28992",
    )
)

Setup the basins:

In [ ]:
profile = pd.DataFrame(
    data={
        "node_id": [2, 2, 5, 5, 12, 12],
        "area": 300_000.0,
        "level": 3 * [0.0, 1.0],
    }
)

state = pd.DataFrame(data={"node_id": [2, 5, 12], "level": 1.0})

basin = ribasim.Basin(profile=profile, state=state)

Setup the flow boundary:

In [ ]:
flow_boundary = ribasim.FlowBoundary(
    static=pd.DataFrame(
        data={
            "node_id": [1],
            "flow_rate": 2.0,
        }
    )
)

Setup the linear resistance:

In [ ]:
linear_resistance = ribasim.LinearResistance(
    static=pd.DataFrame(
        data={
            "node_id": [4],
            "resistance": 0.06,
        }
    )
)

Setup the tabulated rating curve:

In [ ]:
tabulated_rating_curve = ribasim.TabulatedRatingCurve(
    static=pd.DataFrame(
        data={
            "node_id": 7,
            "level": [0.0, 0.5, 1.0],
            "flow_rate": [0.0, 0.0, 2.0],
        }
    )
)

Setup the fractional flow:

In [ ]:
fractional_flow = ribasim.FractionalFlow(
    static=pd.DataFrame(
        data={
            "node_id": [8, 8, 9, 9],
            "fraction": [0.6, 0.9, 0.4, 0.1],
            "control_state": ["divert", "close", "divert", "close"],
        }
    )
)

Setup the terminal:

In [ ]:
terminal = ribasim.Terminal(
    static=pd.DataFrame(
        data={
            "node_id": [10],
        }
    )
)

Setup the discrete control:

In [ ]:
condition = pd.DataFrame(
    data={
        "node_id": [11],
        "listen_feature_id": 5,
        "variable": "level",
        "greater_than": 0.52,
    }
)

logic = pd.DataFrame(
    data={
        "node_id": 11,
        "truth_state": ["T", "F"],
        "control_state": ["divert", "close"],
    }
)

discrete_control = ribasim.DiscreteControl(condition=condition, logic=logic)

Setup the users:

In [ ]:
user = ribasim.User(
    static=pd.DataFrame(
        data={
            "node_id": [6, 13],
            "demand": [1.5, 1.0],
            "return_factor": 0.0,
            "min_level": -1.0,
            "priority": [1, 3],
        }
    ),
    time=pd.DataFrame(
        data={
            "node_id": [3, 3, 3, 3],
            "demand": [0.0, 1.0, 1.2, 1.2],
            "priority": [1, 1, 2, 2],
            "return_factor": 0.0,
            "min_level": -1.0,
            "time": 2 * ["2020-01-01 00:00:00", "2020-01-20 00:00:00"],
        }
    ),
)

Setup the allocation:

In [ ]:
allocation = ribasim.Allocation(use_allocation=True, timestep=86400)

Setup a model:

In [ ]:
model = ribasim.Model(
    network=ribasim.Network(
        node=node,
        edge=edge,
    ),
    basin=basin,
    flow_boundary=flow_boundary,
    linear_resistance=linear_resistance,
    tabulated_rating_curve=tabulated_rating_curve,
    terminal=terminal,
    user=user,
    discrete_control=discrete_control,
    fractional_flow=fractional_flow,
    allocation=allocation,
    starttime="2020-01-01 00:00:00",
    endtime="2020-01-20 00:00:00",
)

Let's take a look at the model:

In [ ]:
model.plot()

Write the model to a TOML and GeoPackage:

In [ ]:
datadir = Path("data")
model.write(datadir / "allocation_example/ribasim.toml")

In [ ]:
# | include: false
from subprocess import run

run(
    [
        "julia",
        "--project=../../core",
        "--eval",
        f'using Ribasim; Ribasim.main("{datadir.as_posix()}/allocation_example/ribasim.toml")',
    ],
    check=True,
)

Now run the model with `ribasim allocation_example/ribasim.toml`.
After running the model, read back the results:

In [ ]:
import matplotlib.ticker as plticker

df_allocation = pd.read_feather(datadir / "allocation_example/results/allocation.arrow")
df_allocation_wide = df_allocation.pivot_table(
    index="time",
    columns=["node_type", "node_id", "priority"],
    values=["demand", "allocated", "realized"],
)
df_allocation_wide = df_allocation_wide.loc[:, (df_allocation_wide != 0).any(axis=0)]

fig, axs = plt.subplots(1, 3, figsize=(8, 5))
fig.suptitle(f"Objective type: {model.allocation.objective_type}", fontweight="bold")

df_allocation_wide["demand"].plot(ax=axs[0], ls=":")
df_allocation_wide["allocated"].plot(ax=axs[1], ls="--")
df_allocation_wide["realized"].plot(ax=axs[2])

fig.tight_layout()
loc = plticker.MultipleLocator(2)

axs[0].set_ylabel("level [m]")

for ax, title in zip(axs, ["Demand", "Allocated", "Abstracted"]):
    ax.set_title(title)
    ax.set_ylim(0.0, 1.6)
    ax.xaxis.set_major_locator(loc)

Some things to note about this plot:

- Abstraction behaves somewhat erratically at the start of the simulation. This is because allocation is based on flows computed in the physical layer, and at the start of the simulation these are not known yet.

- Although there is a plotted line for abstraction per priority, abstraction is actually accumulated over all priorities per user.

In [ ]:
df_basin = pd.read_feather(datadir / "allocation_example/results/basin.arrow")
df_basin_wide = df_basin.pivot_table(
    index="time", columns="node_id", values=["storage", "level"]
)

ax = df_basin_wide["level"].plot()
ax.set_title("Basin levels")
ax.set_ylabel("level [m]")

# Model with allocation (basin supply/demand)

Setup the nodes:

In [ ]:
xy = np.array(
    [
        (0.0, 0.0),  # 1: FlowBoundary
        (1.0, 0.0),  # 2: Basin
        (2.0, 0.0),  # 3: User
        (1.0, -1.0),  # 4: LevelDemand
        (2.0, -1.0),  # 5: Basin
    ]
)
node_xy = gpd.points_from_xy(x=xy[:, 0], y=xy[:, 1])

node_type = ["FlowBoundary", "Basin", "User", "LevelDemand", "Basin"]

# Make sure the feature id starts at 1: explicitly give an index.
node = ribasim.Node(
    df=gpd.GeoDataFrame(
        data={
            "type": node_type,
            "subnetwork_id": 5 * [2],
        },
        index=pd.Index(np.arange(len(xy)) + 1, name="fid"),
        geometry=node_xy,
        crs="EPSG:28992",
    )
)

Setup the edges:

In [ ]:
from_id = np.array([1, 2, 4, 3, 4])
to_id = np.array([2, 3, 2, 5, 5])
edge_type = ["flow", "flow", "control", "flow", "control"]
subnetwork_id = [2, None, None, None, None]

lines = node.geometry_from_connectivity(from_id.tolist(), to_id.tolist())
edge = ribasim.Edge(
    df=gpd.GeoDataFrame(
        data={
            "from_node_id": from_id,
            "to_node_id": to_id,
            "edge_type": edge_type,
            "subnetwork_id": subnetwork_id,
        },
        geometry=lines,
        crs="EPSG:28992",
    )
)

Setup the basins:

In [ ]:
profile = pd.DataFrame(
    data={"node_id": [2, 2, 5, 5], "area": 1e3, "level": [0.0, 1.0, 0.0, 1.0]}
)
static = pd.DataFrame(
    data={
        "node_id": [5],
        "drainage": 0.0,
        "potential_evaporation": 0.0,
        "infiltration": 0.0,
        "precipitation": 0.0,
        "urban_runoff": 0.0,
    }
)
time = pd.DataFrame(
    data={
        "node_id": 2,
        "time": ["2020-01-01 00:00:00", "2020-01-16 00:00:00"],
        "drainage": 0.0,
        "potential_evaporation": 0.0,
        "infiltration": 0.0,
        "precipitation": [1e-6, 0.0],
        "urban_runoff": 0.0,
    },
)

state = pd.DataFrame(data={"node_id": [2, 5], "level": 0.5})
basin = ribasim.Basin(profile=profile, static=static, time=time, state=state)

Setup the flow boundary:

In [ ]:
flow_boundary = ribasim.FlowBoundary(
    static=pd.DataFrame(data={"node_id": [1], "flow_rate": 1e-3})
)

Setup allocation level control:

In [ ]:
level_demand = ribasim.LevelDemand(
    static=pd.DataFrame(
        data={"node_id": [4], "priority": 1, "min_level": 1.0, "max_level": 1.5}
    )
)

Setup the users:

In [ ]:
user = ribasim.User(
    static=pd.DataFrame(
        data={
            "node_id": [3],
            "priority": [2],
            "demand": [1.5e-3],
            "return_factor": [0.2],
            "min_level": [0.2],
        }
    )
)

Setup the allocation:

In [ ]:
allocation = ribasim.Allocation(use_allocation=True, timestep=1e5)

Setup a model:

In [ ]:
model = ribasim.Model(
    network=ribasim.Network(node=node, edge=edge),
    basin=basin,
    flow_boundary=flow_boundary,
    level_demand=level_demand,
    user=user,
    allocation=allocation,
    starttime="2020-01-01 00:00:00",
    endtime="2020-02-01 00:00:00",
)

Let's take a look at the model:

In [ ]:
model.plot()

Write the model to a TOML and GeoPackage:

In [ ]:
model.write(datadir / "level_demand/ribasim.toml")

In [ ]:
# | include: false
from subprocess import run

run(
    [
        "julia",
        "--project=../../core",
        "--eval",
        f'using Ribasim; Ribasim.main("{datadir.as_posix()}/level_demand/ribasim.toml")',
    ],
    check=True,
)

Now run the model with `ribasim level_demand/ribasim.toml`.
After running the model, read back the results:

In [ ]:
df_basin = pd.read_feather(datadir / "level_demand/results/basin.arrow")
df_basin = df_basin[df_basin.node_id == 2]
df_basin_wide = df_basin.pivot_table(
    index="time", columns="node_id", values=["storage", "level"]
)
ax = df_basin_wide["level"].plot()
where_allocation = (
    df_basin_wide.index - df_basin_wide.index[0]
).total_seconds() % model.allocation.timestep == 0
where_allocation[0] = False
df_basin_wide[where_allocation]["level"].plot(
    style="o",
    ax=ax,
)
ax.set_ylabel("level [m]")

In the plot above, the line denotes the level of basin #2 over time and the dots denote the times at which allocation optimization was run, with intervals of $\Delta t_{\text{alloc}}$. The basin level is a piecewise linear function of time, with several stages explained below.

Constants:
- $d$: user #3 demand,
- $\phi$: precipitation rate on basin #2,
- $q$: flow of the level boundary.

Stages:
- In the first stage the user abstracts fully, so the net change of basin #2 is $q + \phi - d$;
- In the second stage the basin takes precedence so the user doesn't abstract, hence the net change of basin #2 is $q + \phi$;
-  In the third stage (and following stages) the basin has no longer a positive demand, since precipitation provides enough water to get the basin to its target level. The FlowBoundary flow gets fully allocated to the user, hence the net change of basin #2 is $\phi$;
- In the fourth stage the basin enters its surplus stage, even though initially the level is below the maximum level. This is because the simulation anticipates that the current precipitation is going to bring the basin level over its maximum level. The net change of basin #2 is now $q + \phi - d$;
- At the start of the fifth stage the precipitation stops, and so the user partly uses surplus water from the basin to fulfill its demand. The net change of basin #2 becomes $q - d$.
- In the final stage the basin is in a dynamical equilibrium, since the basin has no supply so the user abstracts precisely the flow from the level boundary.